# 동적 웹크롤링

### 라이브러리 설치

In [ ]:
!pip install selenium
!pip install bs4
!pip install tqdm

## WebDriver 임포트

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# 객체생성
wd = webdriver.Chrome()   # 자동화 브라우저 생성

In [ ]:
# 자동화 브라우져에 주소를 입력
wd.get("https://www.coffeebeankorea.com/store/store.asp")

In [ ]:
# 해당 주소에 있는 자바스크립트 실행
wd.execute_script("storePop2(0)")

In [ ]:
# html 소스로 변환
html = wd.page_source

In [ ]:
# 파싱을 위해 beautifulsoup 객체 생성
soupCB1 = BeautifulSoup(html,'html.parser')

In [ ]:
# html 확인 - 필요하면
# print(soupCB1.prettify())

In [ ]:
# 추출하려는 패턴을 확인하고 적용  매장명
store_title = soupCB1.select('div.store_txt>h2')
len(store_title) > 0

In [ ]:
# 매장정보
tr_lists = soupCB1.select('table.store_table>tbody>tr')

In [ ]:
print(tr_lists[0].th.text,  tr_lists[0].td.text)
print(tr_lists[1].th.text,  tr_lists[1].td.text)
print(tr_lists[2].th.text,  tr_lists[2].td.text)
print(tr_lists[3].th.text,  tr_lists[3].td.text)
print(tr_lists[4].th.text,  tr_lists[4].td.text)

# 한꺼번에 실행하기

In [173]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm # 진행상태를 표시
import pandas as pd

In [174]:
# 동적페이지 크롤링
wd = webdriver.Chrome()
store_lists = []
for store_num in tqdm(range(5)):
    try:
        wd.get("https://www.coffeebeankorea.com/store/store.asp")
        time.sleep(1)  # 대기
        
        wd.execute_script(f"storePop2({store_num})")
        time.sleep(1)
        
        html = wd.page_source
        soupCB1 = BeautifulSoup(html,'html.parser')
        store_title = soupCB1.select('div.store_txt>h2')[0].text
        if store_title == '':
            continue
        # 매장정보
        tr_lists = soupCB1.select('table.store_table>tbody>tr')    
        store_lists.append([store_title, tr_lists[0].td.text, tr_lists[1].td.text, tr_lists[2].td.text, tr_lists[3].td.text,tr_lists[4].td.text])       
        
        # # 버튼요소를 찾기  닫으려는 대상 객체가 없으면 예외가발생
        # button = WebDriverWait(wd,1).until(
        #     EC.presence_of_all_elements_located( (By.ID,'closeBtn1ActiveBtn'))
        # )[0]
        # button.click()
    except Exception as e:
        pass
        
# 매장정보 추출        
df = pd.DataFrame(store_lists,columns=['매장명','영업시간','주차','주소','전화번호','기타정보'])
df.to_csv('coffebean_store_lists.csv', encoding='cp949

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:14<00:00,  2.83s/it]


,매장명,영업시간,주차,주소,전화번호,기타정보
0,차병원점,평일 07:00~18:00 | 토요일 07:00~15:00 | 일.공휴일 휴무,병원주차장 이용가능 (병원방문시 무료),서울시 강남구 논현로 566 강남차병원1층,02-538-7615,식약처 인증 위생등급 매우 우수매장
1,차,병,원,점,None,None


In [175]:
store_lists

[['차병원점',
  ' 평일 07:00~18:00 | 토요일 07:00~15:00 | 일.공휴일 휴무',
  '병원주차장 이용가능 (병원방문시 무료)',
  '서울시 강남구 논현로 566 강남차병원1층  ',
  '02-538-7615',
  '식약처 인증 위생등급 매우 우수매장'],
 '차병원점']

# 예외처리

In [ ]:
for i in ['1','2','abc','3']:
    try:
        print(int(i))
    except Exception as e:
        print("예외가 발생했습니다.", e)

In [ ]:
for i in ['1','2','abc','3']:    
        print(int(i))
    